# Model I/O - Prompt ExampleSelector 少样本提示，示例选择器

## LengthBasedExampleSelector 按长度示例选择器

引入类

In [ ]:
from langchain import PromptTemplate
from langchain.prompts import FewShotPromptTemplate

### 1、创建一个少样例提示

In [ ]:
# 定义示例
examples = [
    {"title": "今日份的小确幸", "content": "早上的一杯咖啡，让忙碌的生活暂时按下暂停键。在这个小小的瞬间，找到了自己的小确幸。"},
    {"title": "一个人的西藏", "content": "独自一人踏上西藏之旅，每一步都是风景，每一刻都是故事。这里的蓝天白云，让我忘记了世界的喧嚣。"},
    {"title": "夏日防晒必备良品", "content": "分享我这个夏天最爱的防晒霜，轻薄不油腻，让我在炎炎夏日也能享受阳光而不畏惧。"}
]

example_prompt = PromptTemplate(
    input_variables=["title", "content"], template="标题: {title}\n内容：{content}"
)

# print(example_prompt.format(**examples[0]))

# 创建 FewShotPromptTemplate
fewShotprompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    example_separator="\n\n",
    prefix="想要创作出具有小红书风格的内容，请参考以下示例：",
    suffix="根据上述示例，尝试创作一个{user_input}。",
    input_variables=["user_input"]
)

# 生成提示
print(fewShotprompt.format(user_input="新的故事"))